# Spatial regression

> **NOTE**: some of this material has been ported and adapted from the PySAL/spreg code for Chapter 5-13 in *Modern Spatial Econometrics in Practice: A Guide to GeoDa, GeoDaSpace and PySAL* by Luc Anselin and Sergio J. Rey. (c) 2014 Luc Anselin and Sergio J. Rey, All Rights Reserved.

* [Basic OLS](#Basic-OLS)
    * Nonspatial diagnostics
    * Spatial diagnostics - spatial specification
* [Spatial Lag Model](#Spatial-Lag-Model)
    * Spatial two Stage Least Squares (S2SLS)
        * Spatial diagnostics
        * Interpretation
            * Direct impact
            * Indirect impact
* [Spatial Model with Lag and Error](#Spatial-Model-with-Lag-and-Error)
    
* [Spatial error model](#Spatial-Error-Model)

## Basic Regression Setup##

**Creating arrays for y and x using the Baltimore house price example**

Preliminaries, importing **numpy** and **PySAL**

In [ ]:
%pylab inline

In [ ]:
import numpy as np
import pysal as ps

Loading the data set **baltim.dbf** from pysal exmaple datasets and creating the data object.

In [ ]:
db = ps.open(ps.examples.get_path("baltim.dbf"),'r')
db.header

Check how many observations:

In [ ]:
len(db)

**y** - dependent variable is PRICE

In [ ]:
y_name = "PRICE"

In [ ]:
from pysal.contrib.viz import mapping as maps
db_pd = ps.pdio.read_files(ps.examples.get_path("baltim.shp"))
fig, ax = plt.subplots(1, figsize=(6, 6))
maps.geoplot(db_pd, y_name,ax=ax, palette='OrRd',classi="Equal_Interval")

Create the y array as a n by 1 column vector (hence the transpose **T**)

In [ ]:
y = np.array([db.by_col(y_name)]).T

Check on the dimensions

In [ ]:
y.shape

**x** - the explanatory variables

First create a list with the variable names, then use a list comprehension to create the **x** array

In [ ]:
x_names = ['NROOM','NBATH','PATIO','FIREPL','AC','GAR','AGE',
           'LOTSZ','SQFT']
x = np.array([db.by_col(var) for var in x_names]).T

Check on dimensions

In [ ]:
x.shape

**Model weights** - needed for spatial diagnostics

k nearest neighbor with k=4 constructed from **baltim.shp**, using STATION as the ID variable

In [ ]:
w = ps.knnW_from_shapefile(ps.examples.get_path('baltim.shp'),k=4,idVariable='STATION')

Quick check on dimension

In [ ]:
w.n

In [ ]:
w.histogram

row-standardize - is **always** necessary

In [ ]:
w.transform = 'r'

Quick check on the values of the weights

In [ ]:
w.weights[1]

## Basic OLS 

**OLS with variable and data set names**

$$\mathbf{y}=\mathbf{X} \beta + \mathbf{\epsilon}$$

In [ ]:
ols1a = ps.spreg.OLS(y,x,name_y=y_name,name_x=x_names, name_ds='baltim.shp')

In [ ]:
print(ols1a.summary)

## OLS with White Test

set **white_test = True**

In [ ]:
ols2 = ps.spreg.OLS(y,x,white_test=True,name_y=y_name,name_x=x_names,name_ds='baltim.shp')

In [ ]:
print( ols2.summary )

now the **White** test is included as one of the diagnostics

## OLS with Spatial Diagnostics

specify the weights (**w**) and set **spat_diag = True** and **moran = True**

specify a name for the weights in **name_w**, all the rest is as before

for convenience, white_test is back to default of False

In [ ]:
ols3 = ps.spreg.OLS(y,x,w=w,spat_diag=True,moran=True,
                       name_y=y_name,name_x=x_names,name_w='baltim_k4',
                       name_ds='baltim.shp')

In [ ]:
print( ols3.summary )

diagnostics for spatial dependence at the bottom of the listing

Pointing to a spatial lag or a SARMA specification? Let's run both of them

## Spatial Lag Model
$$\mathbf{y}= \rho \mathbf{Wy} + \mathbf{X} \beta + \mathbf{\epsilon}$$

### Spatial two Stage Least Squares (S2SLS)

The endogeneity of the lagged dependent variable $\mathbf{Wy}$ can be dealt with by using instrumental variables $[\mathbf{X}, \mathbf{WX}, \mathbf{W^2X}]$.

The default setting of instrumental variables in PySAL `GM_Lag` includes exogeneous variables and their first-oder spatial lags $[\mathbf{X}, \mathbf{WX}]$.

In [ ]:
S2SLS1 = ps.spreg.GM_Lag(y,x,w=w,name_y=y_name,name_x=x_names,name_w='baltim_k4',name_ds='baltim')

In [ ]:
print( S2SLS1.summary )

#### including spatial diagnostics, set `spat_diag=True`

In [ ]:
S2SLS1_sp = ps.spreg.GM_Lag(y,x,w=w,spat_diag=True,
                          name_y=y_name,name_x=x_names,
                          name_w='baltim_k4',name_ds='baltim')
print( S2SLS1_sp.summary )

### Direct & Indirect Impacts

In [ ]:
b = S2SLS1_sp.betas[:-1]
b

In [ ]:
rho = S2SLS1_sp.betas[-1]
rho

In [ ]:
btot = b / (1.0 - rho) #direct impact
bind = btot - b #indirect impact

In [ ]:
varnames = ["CONSTANT"] + x_names
print( "Variable       Direct       Indirect      Total" )
for i in range(len(varnames)):
    print("%10s %12.7f %12.7f %12.7f" % (varnames[i],b[i][0],bind[i][0],btot[i][0]))

#### using second order spatial lags for the instruments, set `w_lags = 2`

In [ ]:
S2SLS2_sp = ps.spreg.GM_Lag(y,x,w=w,w_lags=2,spat_diag=True,
                          name_y=y_name,name_x=x_names,
                          name_w='baltim_k4',name_ds='baltim')
print(S2SLS2_sp.summary)

## Spatial Model with Lag and Error
Or SAR-SAR

$$\mathbf{y}= \rho \mathbf{Wy} + \mathbf{X} \beta + \mathbf{\mu}$$
$$\mathbf{\mu} = \lambda \mathbf{W \mu} + \mathbf{\epsilon}$$

In [ ]:
ps.spreg.GM_Combo?

In [ ]:
combo1 = ps.spreg.GM_Combo(y,x,w=w,name_y=y_name,
                       name_x=x_names,name_w="baltim_k4",
                       name_ds="baltim")
print(combo1.summary)

## Spatial Error Model

$$\mathbf{y}= \mathbf{X} \beta + \mathbf{\mu}$$
$$\mathbf{\mu} = \lambda \mathbf{W \mu} + \mathbf{\epsilon}$$

In [ ]:
sem = ps.spreg.GM_Error(y,x,w=w,name_y=y_name,
                       name_x=x_names,name_w="baltim_k4",
                       name_ds="baltim")
print( sem.summary )

## Practice##

Create a regression object using the classic Harrison-Rubinfeld Boston house price data set (included as Boston.shp). Regression median house value (MEDV) on crime rate (CRIM), Charles river dummy (CHAS), nitric oxides (NOX), number of rooms (RM), age (AGE), weighted distance to five employment centers (DIS) and percent "lower status population" (LSTAT). A full description of the Boston data set is available on the GeoDa Center sample data set site.

Use a k-nearest neighbor spatial weights (k = 4) for the spatial diagnostics.

What is the most likely alternative spatial regression model, given the results of a spatial specification search.

Try any other combinations of explanatory variables, spatial weights and standard error specifications.

## Solutions

In [ ]:
db = ps.open("data/Boston.dbf",'r')
db.header

In [ ]:
y_name = "MEDV"
y = np.array([db.by_col(y_name)]).T

In [ ]:
x_names = ["CRIM","CHAS","NOX","RM","AGE","DIS","LSTAT"]
x = np.array([db.by_col(var) for var in x_names]).T

In [ ]:
w = ps.knnW_from_shapefile('data/Boston.shp',k=4)

In [ ]:
ols1a = ps.spreg.OLS(y,x,w=w,spat_diag=True,moran=True, name_y=y_name,name_x=x_names, name_ds='Boston.shp')

In [ ]:
print(ols1a.summary)

In [ ]:
kw = ps.adaptive_kernelW_from_shapefile('data/Boston.shp',
                                             k=12,diagonal=True)

In [ ]:
ols6 = ps.spreg.OLS(y,x,w=w,spat_diag=True,moran=True,robust='hac',gwk=kw,
                       name_y=y_name,name_x=x_names,
                       name_gwk='Boston_tri_k12',name_ds='Boston.shp')
print(ols6.summary)